In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
import pandas as pd
import numpy as np
import lightgbm as lgb

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [0]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [0]:
all = pd.concat([train, test], axis =0).reset_index(drop=True)
all

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,A/5 21171
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1.0,PC 17599
2,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,1.0,STON/O2. 3101282
3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,1.0,113803
4,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,0,0.0,373450
5,NaN,NaN,Q,8.4583,"Moran, Mr. James",0,6,3,male,0,0.0,330877
6,54.0,E46,S,51.8625,"McCarthy, Mr. Timothy J",0,7,1,male,0,0.0,17463
7,2.0,NaN,S,21.0750,"Palsson, Master. Gosta Leonard",1,8,3,male,3,0.0,349909
8,27.0,NaN,S,11.1333,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",2,9,3,female,0,1.0,347742
9,14.0,NaN,C,30.0708,"Nasser, Mrs. Nicholas (Adele Achem)",0,10,2,female,1,1.0,237736


In [0]:
def kesson_table(df):
    null_val = df.isnull().sum()
    percent = 100 * df.isnull().sum()/len(df)
    kesson_table = pd.concat([null_val, percent], axis=1)
    kesson_table_ren_columns = kesson_table.rename(
        columns = {0 : '欠損数', 1 : '%'})
    return kesson_table_ren_columns

kesson_table(all)

,欠損数,%
Age,263,20.091673
Cabin,1014,77.463713
Embarked,2,0.152788
Fare,1,0.076394
Name,0,0.000000
Parch,0,0.000000
PassengerId,0,0.000000
Pclass,0,0.000000
Sex,0,0.000000
SibSp,0,0.000000


In [0]:
all["surname"] = all["Name"].apply(lambda x:x.split(",")[0])

In [0]:
cols = ["surname", "Sex","Embarked"]
all_to_cat = pd.get_dummies(all, columns=cols)
print(all_to_cat.shape)

(1309, 890)


In [0]:
data_counts = all['surname'].value_counts()
surname_to_drop = data_counts[data_counts < 4].index.tolist()

to_drop_list = []
target_col = "surname"

for each_data in surname_to_drop:
    col_name = "{}_{}".format(target_col, each_data)
    to_drop_list.append(col_name)
    
all_to_cat.drop(columns=to_drop_list, inplace=True)
print(all_to_cat.shape)

(1309, 57)


In [0]:
all_to_cat.head()

,Age,Cabin,Fare,Name,Parch,PassengerId,Pclass,SibSp,Survived,Ticket,surname_Allison,surname_Andersson,surname_Asplund,surname_Baclini,surname_Becker,surname_Boulos,surname_Brown,surname_Cacic,surname_Carter,surname_Davies,surname_Dean,surname_Elias,surname_Ford,surname_Fortune,surname_Goldsmith,surname_Goodwin,surname_Gustafsson,surname_Hansen,surname_Harper,surname_Harris,surname_Hart,surname_Herman,surname_Hocking,surname_Johansson,surname_Johnson,surname_Johnston,surname_Kelly,surname_Laroche,surname_Lefebre,surname_Olsen,surname_Palsson,surname_Panula,surname_Rice,surname_Ryerson,surname_Sage,surname_Skoog,surname_Smith,surname_Thomas,surname_Vander Planke,surname_Ware,surname_West,surname_Williams,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,22.0,NaN,7.2500,"Braund, Mr. Owen Harris",0,1,3,1,0.0,A/5 21171,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
1,38.0,C85,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,1,1.0,PC 17599,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
2,26.0,NaN,7.9250,"Heikkinen, Miss. Laina",0,3,3,0,1.0,STON/O2. 3101282,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
3,35.0,C123,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,1,1.0,113803,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
4,35.0,NaN,8.0500,"Allen, Mr. William Henry",0,5,3,0,0.0,373450,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1


In [0]:
to_drop = ["PassengerId", "Name", "Ticket", "Cabin"]

all_to_cat.drop(columns=to_drop, inplace=True)
print(all_to_cat.shape)

(1309, 53)


In [0]:
all_to_cat['Age'].fillna(all_to_cat['Age'].mode()[0], inplace=True)

In [0]:
all_to_cat

,Age,Fare,Parch,Pclass,SibSp,Survived,surname_Allison,surname_Andersson,surname_Asplund,surname_Baclini,surname_Becker,surname_Boulos,surname_Brown,surname_Cacic,surname_Carter,surname_Davies,surname_Dean,surname_Elias,surname_Ford,surname_Fortune,surname_Goldsmith,surname_Goodwin,surname_Gustafsson,surname_Hansen,surname_Harper,surname_Harris,surname_Hart,surname_Herman,surname_Hocking,surname_Johansson,surname_Johnson,surname_Johnston,surname_Kelly,surname_Laroche,surname_Lefebre,surname_Olsen,surname_Palsson,surname_Panula,surname_Rice,surname_Ryerson,surname_Sage,surname_Skoog,surname_Smith,surname_Thomas,surname_Vander Planke,surname_Ware,surname_West,surname_Williams,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,22.0,7.2500,0,3,1,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
1,38.0,71.2833,0,1,1,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
2,26.0,7.9250,0,3,0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
3,35.0,53.1000,0,1,1,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
4,35.0,8.0500,0,3,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
5,24.0,8.4583,0,3,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
6,54.0,51.8625,0,1,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
7,2.0,21.0750,1,3,3,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
8,27.0,11.1333,2,3,0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
9,14.0,30.0708,0,2,1,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0


In [0]:
train_to_cat = all_to_cat.iloc[0:891, :]
train_to_cat.head()

,Age,Fare,Parch,Pclass,SibSp,Survived,surname_Allison,surname_Andersson,surname_Asplund,surname_Baclini,surname_Becker,surname_Boulos,surname_Brown,surname_Cacic,surname_Carter,surname_Davies,surname_Dean,surname_Elias,surname_Ford,surname_Fortune,surname_Goldsmith,surname_Goodwin,surname_Gustafsson,surname_Hansen,surname_Harper,surname_Harris,surname_Hart,surname_Herman,surname_Hocking,surname_Johansson,surname_Johnson,surname_Johnston,surname_Kelly,surname_Laroche,surname_Lefebre,surname_Olsen,surname_Palsson,surname_Panula,surname_Rice,surname_Ryerson,surname_Sage,surname_Skoog,surname_Smith,surname_Thomas,surname_Vander Planke,surname_Ware,surname_West,surname_Williams,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,22.0,7.2500,0,3,1,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
1,38.0,71.2833,0,1,1,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
2,26.0,7.9250,0,3,0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
3,35.0,53.1000,0,1,1,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
4,35.0,8.0500,0,3,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1


In [0]:
train_train = train_to_cat.drop(["Survived"], axis=1)
train_train.head()

,Age,Fare,Parch,Pclass,SibSp,surname_Allison,surname_Andersson,surname_Asplund,surname_Baclini,surname_Becker,surname_Boulos,surname_Brown,surname_Cacic,surname_Carter,surname_Davies,surname_Dean,surname_Elias,surname_Ford,surname_Fortune,surname_Goldsmith,surname_Goodwin,surname_Gustafsson,surname_Hansen,surname_Harper,surname_Harris,surname_Hart,surname_Herman,surname_Hocking,surname_Johansson,surname_Johnson,surname_Johnston,surname_Kelly,surname_Laroche,surname_Lefebre,surname_Olsen,surname_Palsson,surname_Panula,surname_Rice,surname_Ryerson,surname_Sage,surname_Skoog,surname_Smith,surname_Thomas,surname_Vander Planke,surname_Ware,surname_West,surname_Williams,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,22.0,7.2500,0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
1,38.0,71.2833,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
2,26.0,7.9250,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
3,35.0,53.1000,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
4,35.0,8.0500,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1


In [0]:
target = train_to_cat["Survived"].values
features = train_train.values

In [0]:
x_train, x_test, t_train, t_test = train_test_split(features, target, test_size =0.3, random_state = 0)

In [0]:
lgb.LGBMClassifier()

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [0]:
train_data = lgb.Dataset(x_train, label=t_train)
eval_data = lgb.Dataset(x_test, label=t_test, reference=train_data)

In [0]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'num_class': 1,
    'verbose': 2,
}

In [0]:
gbm = lgb.train(
    params,
    train_data,
    valid_sets=eval_data,
    num_boost_round=500,
    verbose_eval=20,
)

[20]	valid_0's binary_logloss: 0.414487
[40]	valid_0's binary_logloss: 0.409904
[60]	valid_0's binary_logloss: 0.429595
[80]	valid_0's binary_logloss: 0.437748
[100]	valid_0's binary_logloss: 0.445584
[120]	valid_0's binary_logloss: 0.456687
[140]	valid_0's binary_logloss: 0.460203
[160]	valid_0's binary_logloss: 0.471113
[180]	valid_0's binary_logloss: 0.480775
[200]	valid_0's binary_logloss: 0.482277
[220]	valid_0's binary_logloss: 0.488184
[240]	valid_0's binary_logloss: 0.494501
[260]	valid_0's binary_logloss: 0.500529
[280]	valid_0's binary_logloss: 0.510563
[300]	valid_0's binary_logloss: 0.520025
[320]	valid_0's binary_logloss: 0.531214
[340]	valid_0's binary_logloss: 0.540603
[360]	valid_0's binary_logloss: 0.549551
[380]	valid_0's binary_logloss: 0.556816
[400]	valid_0's binary_logloss: 0.568494
[420]	valid_0's binary_logloss: 0.57825
[440]	valid_0's binary_logloss: 0.586598
[460]	valid_0's binary_logloss: 0.594341
[480]	valid_0's binary_logloss: 0.605491
[500]	valid_0's binar

loss値上がってしまっている

In [0]:
test_to_cat = all_to_cat.iloc[891:, :]
test_to_cat.head()

,Age,Fare,Parch,Pclass,SibSp,Survived,surname_Allison,surname_Andersson,surname_Asplund,surname_Baclini,surname_Becker,surname_Boulos,surname_Brown,surname_Cacic,surname_Carter,surname_Davies,surname_Dean,surname_Elias,surname_Ford,surname_Fortune,surname_Goldsmith,surname_Goodwin,surname_Gustafsson,surname_Hansen,surname_Harper,surname_Harris,surname_Hart,surname_Herman,surname_Hocking,surname_Johansson,surname_Johnson,surname_Johnston,surname_Kelly,surname_Laroche,surname_Lefebre,surname_Olsen,surname_Palsson,surname_Panula,surname_Rice,surname_Ryerson,surname_Sage,surname_Skoog,surname_Smith,surname_Thomas,surname_Vander Planke,surname_Ware,surname_West,surname_Williams,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
891,34.5,7.8292,0,3,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
892,47.0,7.0000,0,3,1,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
893,62.0,9.6875,0,2,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
894,27.0,8.6625,0,3,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
895,22.0,12.2875,1,3,1,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1


In [0]:
test_test = test_to_cat.drop(["Survived"], axis=1)
test_test.head()

,Age,Fare,Parch,Pclass,SibSp,surname_Allison,surname_Andersson,surname_Asplund,surname_Baclini,surname_Becker,surname_Boulos,surname_Brown,surname_Cacic,surname_Carter,surname_Davies,surname_Dean,surname_Elias,surname_Ford,surname_Fortune,surname_Goldsmith,surname_Goodwin,surname_Gustafsson,surname_Hansen,surname_Harper,surname_Harris,surname_Hart,surname_Herman,surname_Hocking,surname_Johansson,surname_Johnson,surname_Johnston,surname_Kelly,surname_Laroche,surname_Lefebre,surname_Olsen,surname_Palsson,surname_Panula,surname_Rice,surname_Ryerson,surname_Sage,surname_Skoog,surname_Smith,surname_Thomas,surname_Vander Planke,surname_Ware,surname_West,surname_Williams,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
891,34.5,7.8292,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
892,47.0,7.0000,0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
893,62.0,9.6875,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
894,27.0,8.6625,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
895,22.0,12.2875,1,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1


In [0]:
test_features = test_test.values

In [0]:
preds = gbm.predict(test_features)
preds

array([4.94072494e-02, 5.14180584e-04, 2.07946605e-04, 9.21356828e-01,
       8.29115117e-01, 5.13828458e-02, 1.03043950e-04, 5.10199569e-02,
       8.97175846e-01, 4.39167672e-08, 1.32547757e-05, 1.64840764e-02,
       9.99954467e-01, 6.52487001e-04, 9.97517468e-01, 9.99837519e-01,
       1.69451527e-02, 6.64637280e-01, 7.50205352e-01, 6.16103957e-03,
       5.13503254e-04, 2.55132512e-01, 9.99999079e-01, 6.45685490e-02,
       9.97756691e-01, 1.53484739e-04, 9.99996804e-01, 8.50376502e-01,
       9.75365311e-01, 4.69460389e-03, 4.11660799e-03, 4.12918202e-03,
       3.13166409e-01, 1.34534416e-02, 7.03179122e-01, 3.39468234e-02,
       4.98559065e-03, 4.65507188e-02, 4.34597071e-03, 4.98922424e-01,
       1.72513647e-03, 5.06293186e-01, 7.47863989e-01, 9.82398881e-01,
       9.99764253e-01, 5.66926054e-04, 3.14492622e-01, 1.30767015e-01,
       9.99493211e-01, 8.94278271e-01, 1.19855866e-01, 9.53667297e-02,
       9.70636985e-01, 9.94487182e-01, 7.89549120e-01, 1.40102036e-06,
      

In [0]:
preds.shape

(418,)

In [0]:
preds01 = np.where(preds < 0.5, 0, 1)
print(preds01)

[0 0 0 1 1 0 0 0 1 0 0 0 1 0 1 1 0 1 1 0 0 0 1 0 1 0 1 1 1 0 0 0 0 0 1 0 0
 0 0 0 0 1 1 1 1 0 0 0 1 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 1 1 0 0 0
 1 1 0 1 0 1 1 0 1 0 0 0 1 1 1 1 0 0 1 0 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0
 1 1 1 1 0 0 1 1 1 1 0 1 0 0 1 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0
 1 0 1 0 0 0 0 0 1 1 1 0 1 1 1 0 0 0 0 0 1 0 0 1 0 0 0 1 1 0 1 1 0 1 1 0 1
 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 0 1 0 0 0 0 1 0 1 0 0 1 0 0 1 0 1 0 1 0
 1 1 1 1 0 1 0 0 0 1 0 0 1 0 0 0 1 1 0 0 1 0 0 0 1 0 1 0 1 0 1 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 1 1 0 1 0 0 0 1 0 0 1 1 0 0 1 0 0 0 0 1 1 0 1 0 0 0 1 0 0
 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 1 0 1 0 0 0 0 1 1 1 1 0 0 1 1 0
 0 1 0 0 1 1 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 1 0 0 1 0
 0 1 1 1 1 0 0 1 0 0 1]


In [0]:
PassengerId = np.array(test["PassengerId"]).astype(int)
                            
my_solution = pd.DataFrame(preds01, PassengerId, columns = ["Survived"])
                            
my_solution.to_csv("FRY1_1.csv", index_label = ["PassengerId"])

In [0]:
from google.colab import files
files.download("FRY1_1.csv")

72.2%。イマイチ。

In [0]:
gbm2 = lgb.train(
    params,
    train_data,
    valid_sets=eval_data,
    num_boost_round=100,
    verbose_eval=5,
)

[5]	valid_0's binary_logloss: 0.516998
[10]	valid_0's binary_logloss: 0.454688
[15]	valid_0's binary_logloss: 0.427422
[20]	valid_0's binary_logloss: 0.414487
[25]	valid_0's binary_logloss: 0.405726
[30]	valid_0's binary_logloss: 0.406917
[35]	valid_0's binary_logloss: 0.410702
[40]	valid_0's binary_logloss: 0.409904
[45]	valid_0's binary_logloss: 0.416515
[50]	valid_0's binary_logloss: 0.420383
[55]	valid_0's binary_logloss: 0.427194
[60]	valid_0's binary_logloss: 0.429595
[65]	valid_0's binary_logloss: 0.433737
[70]	valid_0's binary_logloss: 0.436594
[75]	valid_0's binary_logloss: 0.436584
[80]	valid_0's binary_logloss: 0.437748
[85]	valid_0's binary_logloss: 0.435804
[90]	valid_0's binary_logloss: 0.43965
[95]	valid_0's binary_logloss: 0.443632
[100]	valid_0's binary_logloss: 0.445584


In [0]:
preds2 = gbm2.predict(test_features)
preds2

array([2.72873622e-02, 1.42027384e-02, 7.08004789e-03, 6.34621623e-01,
       5.96275956e-01, 3.38214551e-01, 1.02572690e-01, 2.28846353e-01,
       6.15053238e-01, 3.36504858e-03, 3.98556584e-03, 1.16223660e-01,
       9.93936991e-01, 1.28303723e-02, 9.67523055e-01, 9.68846339e-01,
       1.10464220e-01, 4.35197113e-01, 4.38224063e-01, 5.07077624e-02,
       4.78741863e-02, 4.26303128e-01, 9.97828577e-01, 1.09301599e-01,
       8.46826894e-01, 8.64610026e-03, 9.95960310e-01, 4.78876826e-01,
       8.18266289e-01, 3.16844243e-02, 2.70947020e-02, 1.71662294e-02,
       4.02436072e-01, 1.66780792e-01, 3.85909409e-01, 5.69321938e-02,
       1.47372025e-01, 3.06662124e-01, 6.49243957e-02, 3.77135884e-01,
       3.21302913e-02, 4.42470141e-01, 1.69608448e-01, 8.87804541e-01,
       9.90442994e-01, 2.49623042e-02, 4.09511586e-01, 1.35891895e-01,
       9.64383966e-01, 6.32375987e-01, 1.67428667e-01, 2.50809691e-01,
       8.83089014e-01, 6.82948743e-01, 5.41247152e-01, 1.00320000e-02,
      

In [0]:
preds201 = np.where(preds2 < 0.5, 0, 1)
print(preds201)

[0 0 0 1 1 0 0 0 1 0 0 0 1 0 1 1 0 0 0 0 0 0 1 0 1 0 1 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 1 0 0 0 1 1 0 0 1 1 1 0 0 0 0 1 0 0 0 1 0 1 1 0 1 1 1 0 0 0
 1 1 0 1 0 1 1 0 0 0 0 0 1 1 1 1 0 0 1 0 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0
 1 1 1 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0
 0 0 1 0 0 0 0 0 1 1 1 0 1 1 1 0 0 0 0 0 1 0 0 1 0 0 0 1 1 0 1 1 0 1 1 0 1
 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 1 1 0 0 0 0 1 0 0 0 0 1 0 0 1 0 1 0 1 0
 1 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 1 1 1 1 1 0 0 0 1 0 1 0 1 0 1 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1 0 0 0 1 1 0
 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 1 0 1 0 0 0 0 1 1 1 1 0 0 1 1 0
 0 1 0 0 1 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0
 0 1 1 1 1 0 0 1 0 0 0]


In [0]:
PassengerId = np.array(test["PassengerId"]).astype(int)
                            
my_solution = pd.DataFrame(preds201, PassengerId, columns = ["Survived"])
                            
my_solution.to_csv("FRY1_2.csv", index_label = ["PassengerId"])

In [0]:
from google.colab import files
files.download("FRY1_2.csv")

74.6%。多少向上。

In [53]:
gbm3 = lgb.train(
    params,
    train_data,
    valid_sets=eval_data,
    num_boost_round=500,
    early_stopping_rounds=100,
    verbose_eval=20,
)

Training until validation scores don't improve for 100 rounds.
[20]	valid_0's binary_logloss: 0.414487
[40]	valid_0's binary_logloss: 0.409904
[60]	valid_0's binary_logloss: 0.429595
[80]	valid_0's binary_logloss: 0.437748
[100]	valid_0's binary_logloss: 0.445584
[120]	valid_0's binary_logloss: 0.456687
Early stopping, best iteration is:
[26]	valid_0's binary_logloss: 0.405014


In [57]:
preds3 = gbm3.predict(test_features)
preds3

array([0.05704717, 0.34238607, 0.06873357, 0.299423  , 0.46059493,
       0.44071202, 0.49521249, 0.35979021, 0.72072105, 0.0485595 ,
       0.05086707, 0.17482346, 0.945932  , 0.03168204, 0.93691355,
       0.91040097, 0.18074502, 0.2314063 , 0.32274054, 0.47791205,
       0.2214194 , 0.49666517, 0.93755003, 0.18457783, 0.86343313,
       0.03840145, 0.94726958, 0.2314063 , 0.57913564, 0.15573265,
       0.03840145, 0.05211944, 0.46382837, 0.32504539, 0.54304678,
       0.15267932, 0.52015232, 0.55625728, 0.09830685, 0.2442925 ,
       0.10341585, 0.39103669, 0.06603356, 0.84924725, 0.93691355,
       0.07927321, 0.54978674, 0.16615058, 0.93691355, 0.45711211,
       0.3107698 , 0.26596956, 0.91305301, 0.76526374, 0.31383847,
       0.10136209, 0.04361169, 0.0753323 , 0.08083727, 0.9308806 ,
       0.06257082, 0.20512073, 0.11910224, 0.83269772, 0.26235869,
       0.86627976, 0.85076859, 0.17664988, 0.64490063, 0.86343313,
       0.81146161, 0.06939529, 0.32274054, 0.55530433, 0.93088

In [58]:
preds301 = np.where(preds3 < 0.5, 0, 1)
print(preds301)

[0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 1 0 0 0 0 0 0 1 0 1 0 1 0 1 0 0 0 0 0 1 0 1
 1 0 0 0 0 0 1 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0 1 0 1 1 0 1 1 1 0 0 1
 1 0 0 1 0 1 1 0 0 0 0 0 1 1 1 1 0 0 1 0 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0
 1 1 1 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0
 0 0 1 0 0 0 0 0 1 1 1 0 1 1 1 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0 1 1 0 1 1 0 1
 0 1 0 0 0 0 0 0 0 1 0 1 1 0 1 1 1 1 1 0 0 0 0 1 0 0 0 0 1 0 0 1 0 1 0 1 0
 1 0 1 1 0 1 0 0 1 1 0 0 1 0 0 0 1 1 1 1 0 0 0 0 1 0 1 0 1 0 1 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 1 0 0 0 1 1 0 1 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1 0 0 0 1 1 0
 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 0 0 0 1 0 1 0 0 0 0 1 1 0 1 0 0 0 1 0
 0 1 0 0 1 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 1 0 1 0 0
 0 1 1 1 1 0 0 1 0 0 0]
